CIFAR10 is a 60,000 images dataset where each image is of shape 32* 32 *3 (height, width, depth). 
- Depth =3, because the images are colored - RGB (3 colors). 
- We need to classify images based on 10 classes.
So it's a multi class classification problem in deep learning.
- We will split the data, 50,000 and 10,000 for training and testing respectively. 
- We will follow a Multi-Model approch and then identify which is the best one for our dataset. (starting from self made Dense Neural Networks to Pretrained Neural Networks including Finetuning)


In [ ]:
# Importing Libraries
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

Create a baseline Sequential Model

In [ ]:
def plain_dense_neural_network(height, width, depth, classes):
  model = tf.keras.models.Sequential([
   tf.keras.layers.Flatten(input_shape=(height, width, depth)),
   tf.keras.layers.Dense(1024, activation=tf.nn.relu),
   tf.keras.layers.Dense(512, activation=tf.nn.relu),
   tf.keras.layers.Dense(classes, activation=tf.nn.softmax )                               
  ])
  return model


In [ ]:
def dense_neural_network_with_dropout(height, width, depth, classes):

    model = tf.keras.models.Sequential([
                                        layers.Flatten(input_shape =(height, width,depth)),
                                        layers.Dense(1024, tf.nn.relu),
                                        layers.Dropout(0.2),
                                        layers.Dense(512, tf.nn.relu),
                                        layers.Dropout(0.2),
                                        layers.Dense(10, tf.nn.softmax)
                                      ])
    model.compile(optimizer='adam', loss ='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

Convolution Neural Networks

In [ ]:
def simple_vgg_based_arch_with_maxpool(height, width, depth, classes):
  initial_filters = 32
  filter_shape=(3,3)
  padding = "same"
  pool_size = (2,2)
  dropout_rate = 0.25
  model = tf.keras.models.Sequential([
                                      keras.layers.Conv2D(initial_filters, filter_shape, padding=padding, input_shape=(height, width, depth), activation='relu'),
                                      keras.layers.Conv2D(initial_filters, filter_shape, padding=padding, input_shape=(height, width, depth), activation='relu'),
                                      keras.layers.MaxPooling2D(pool_size),
                                    

                                      keras.layers.Conv2D(64, filter_shape, padding=padding, input_shape=(height, width, depth), activation='relu'),
                                      keras.layers.Conv2D(initial_filters, filter_shape, padding=padding, input_shape=(height, width, depth), activation='relu'),
                                      keras.layers.MaxPooling2D(pool_size),
                                      

                                      keras.layers.Flatten(),
                                      keras.layers.Dense(512, activation='relu'),
                                  

                                      keras.layers.Dense(classes, activation='softmax')

  ])

  return model



In [ ]:
def simple_vgg_based_arch_with_maxpool_and_dropout(height, width, depth, classes):
  initial_filters = 32
  filter_shape=(3,3)
  padding = "same"
  pool_size = (2,2)
  dropout_rate = 0.25
  model = tf.keras.models.Sequential([
                                      keras.layers.Conv2D(initial_filters, filter_shape, padding=padding, input_shape=(height, width, depth), activation='relu'),
                                      keras.layers.Conv2D(initial_filters, filter_shape, padding=padding, input_shape=(height, width, depth), activation='relu'),
                                      keras.layers.MaxPooling2D(pool_size),
                                      keras.layers.SpatialDropout2D(dropout_rate),

                                      keras.layers.Conv2D(64, filter_shape, padding=padding, input_shape=(height, width, depth), activation='relu'),
                                      keras.layers.Conv2D(initial_filters, filter_shape, padding=padding, input_shape=(height, width, depth), activation='relu'),
                                      keras.layers.MaxPooling2D(pool_size),
                                      keras.layers.SpatialDropout2D(dropout_rate),

                                      keras.layers.Flatten(),
                                      keras.layers.Dense(512, activation='relu'),
                                      keras.layers.Dropout(0.5),

                                      keras.layers.Dense(classes, activation='softmax')

  ])

  return model


In [ ]:
def feature_extraction_using_pretrained_vgg16Model(height, width, depth, classes, X_train, X_test):
    
    # creating the base model of pre-trained VGG16 model
    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape =(width, height,depth))
    
    # extracting features for training frames
    X_train = base_model.predict(X_train)
    print(X_train.shape)
    
    # extracting features for validation frames
    X_test = base_model.predict(X_test)
    print(X_test.shape)
    
    # reshaping the training as well as validation frames in single dimension
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)
    
    # Now we have obtained our new dataset, which can be used for another off-the-shelf Classifier like Random Forest
    num_trees = 200
    model = RandomForestClassifier(num_trees)
    
    return model, X_train, X_test
  

In [ ]:
def finetuning(height, width, depth, classes):
    # creating the base model of pre-trained VGG16 model
    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape =(width, height,depth))

    # freeze weights of  layers
    base_model.trainable=False
    
    # Phase A
    model = tf.keras.models.Sequential()
    # add the basemodel to your new Sequential Model
    model.add(base_model)

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dense(classes, activation='softmax'))

    return model


In [ ]:
def resnet():
  pass

In [ ]:
def learning_curves(history):

  plt.style.use('ggplot')
  plt.figure()
  plt.plot(np.arange (0, num_epochs ), history.history['loss'], train_loss)
  plt.plot(np.arange (0, num_epochs ), history.history['val_loss'], val_loss)
  plt.plot(np.arange (0, num_epochs ), history.history['acc'], train_acc)
  plt.plot(np.arange (0, num_epochs ), history.history['val_acc'], val_acc)
  plt.title('Training Loss and Accuracy')
  plt.xlabel('Epoch#')
  plt.ylabel('Loss/Accuracy')
  plt.legend()
  plt.show()


In [ ]:
def visualization():
  pass

In [ ]:
def main():

  # Setting the variables
  NUM_EPOCHS = 50
  height =32
  width =32
  depth=3
  classes =10

  # Loading the data
  cifar = tf.keras.datasets.cifar10
  # print(cifar)

  # Splitting into train and test
  (x_train , y_train),( x_test , y_test ) = cifar.load_data()

  # print(x_train.shape) --50000
  # print(y_train.shape) 
  # print(x_test.shape)-- 10000
  # print(y_test.shape)

  # plt.figure(1, figsize=(3, 3))
  # plt.imshow(x_train.images[3], cmap=plt.cm.gray_r, interpolation='nearest')
  # plt.show()

  #  Normalize the data
  x_train, x_test = x_train / 255.0, x_test / 255
  # print(x_train[0])

  # Models
  choice = 3
  if choice==1:
    # Plain Dense Neural Network
    model = plain_dense_neural_network(height, width, depth, classes)
  
  if choice ==2:
     # Plain Dense Neural Network + Dropout
    model = dense_neural_network_with_dropout(height, width, depth, classes)

  if choice ==3:
    # Simple VGG based Arch with Max Pool
    model = simple_vgg_based_arch_with_maxpool(height, width, depth, classes)
    pass

  if choice ==4:
    model = simple_vgg_based_arch_with_maxpool_and_dropout(height, width, depth, classes)
    # Simple VGG +Max Pool +Dropout - observe the difference of learning curves between choice 2

  if choice ==5:
    # Transfer Learning - Feature Extraction after removing last FC layer
    model, x_train, x_test = feature_extraction_using_pretrained_vgg16Model(height, width, depth, classes, x_train, x_test)
    # Compile the Model
  
    # Fit the Model
    model.fit(x_train , y_train)
    results = model.predict(x_test)
    print(metrics.accuracy_score(results, y_test))
    
  if choice ==6:
    # Transfer Learning - Fine Tuning
    model = finetuning(height, width, depth, classes)

  if choice ==7:
    # Ensembles
    pass  

  # Compile the Model
  model.compile(loss ='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Fit the Model
  history = model.fit(x_train , y_train , NUM_EPOCHS, validation_data = (x_test , y_test))
  
  # Visualize through learning curve
  learning_curves(history)
  # Evaluate the Model

main()

1000/1000 [==============================] - 38s 37ms/step - loss: 2.1139 - accuracy: 0.2778 - val_loss: 1.7465 - val_accuracy: 0.3639


- Model 6 - Finetuning - 1000/1000 [==============================] - 625s 625ms/step - loss: 1.7445 - accuracy: 0.3856 - val_loss: 1.3832 - val_accuracy: 0.5236

- Model 5- Feature Extraction - 0.5349

- Model 4 - VGG with dropout - 1000/1000 [==============================] - 239s 239ms/step - loss: 1.8751 - accuracy: 0.3043 - val_loss: 1.2101 - val_accuracy: 0.5640

- Model 3 - 1000/1000 [==============================] - 240s 239ms/step - loss: 1.5921 - accuracy: 0.4185 - val_loss: 1.0369 - val_accuracy: 0.6236

- Model 6 again - 1000/1000 [==============================] - 638s 637ms/step - loss: 1.7500 - accuracy: 0.3843 - val_loss: 1.4055 - val_accuracy: 0.5139

- Model 2 - Dense with dropout - 1000/1000 [==============================] - 39s 39ms/step - loss: 2.1506 - accuracy: 0.2351 - val_loss: 1.7668 - val_accuracy: 0.3649

- Model 1 - Plain DNN - 1000/1000 [==============================] - 38s 37ms/step - loss: 2.1139 - accuracy: 0.2778 - val_loss: 1.7465 - val_accuracy: 0.3639


References:

https://www.cs.toronto.edu/~kriz/cifar.html